# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [2]:
!pip install gensim

     |████████████████████████████████| 24.7MB 2.2MB/s eta 0:00:01     |██████████████████▊             | 14.4MB 1.5MB/s eta 0:00:07
     |████████████████████████████████| 71kB 776kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73088 sha256=f348e56d98a765e32a8d48f57cb235fdc38a2ad003724922e87d3675a6683b3d
  Stored in directory: /Users/kaan/Library/Caches/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open


In [4]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [12]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
raw_df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [16]:
raw_df['combined_text'] = raw_df.headline+" "+raw_df.short_description
data = raw_df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [17]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [18]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

In [23]:
model.corpus_count

200853

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [22]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55565881, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [24]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [25]:
wv.most_similar('Texas')

[('Ohio', 0.7884379625320435),
 ('Louisiana', 0.7858783006668091),
 ('Connecticut', 0.7831676006317139),
 ('Pennsylvania', 0.7823770046234131),
 ('Oklahoma', 0.7789804935455322),
 ('Illinois', 0.7777242064476013),
 ('Arkansas', 0.7762702703475952),
 ('Maryland', 0.7741101384162903),
 ('Oregon', 0.7725094556808472),
 ('Georgia', 0.771664559841156)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [27]:
wv.most_similar(negative='Texas')

[('Orbuch', 0.3923035264015198),
 ('Animal-rescue', 0.38504117727279663),
 ('Headstrong', 0.37532901763916016),
 ('much-vaunted', 0.37234875559806824),
 ('Chademana', 0.3719084560871124),
 ('once-reliable', 0.3631740212440491),
 ('human-friendly', 0.3596299886703491),
 ('exiling', 0.35632139444351196),
 ('Rank-and-file', 0.3538563847541809),
 ('Late-Deciding', 0.3515014946460724)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [31]:
wv['Texas']

array([-2.53681   ,  1.0627595 ,  0.12207239, -1.1645964 ,  0.7073523 ,
       -1.790685  ,  0.28787345,  0.76303273,  0.51373   , -0.54322803,
       -1.6441362 , -0.3770729 ,  0.74043167,  0.866317  ,  1.5720721 ,
       -0.05699977, -0.31089288,  2.4481623 , -0.5829587 ,  2.7237964 ,
       -1.5651635 , -0.59132856,  0.11079019, -0.03843057, -0.35606778,
       -0.04127937,  2.5609539 ,  0.6765614 ,  1.6640892 ,  0.389934  ,
        3.6540864 , -2.1164572 ,  1.9091955 ,  2.1305652 ,  2.0760612 ,
       -0.21690187,  0.9643949 ,  1.1110433 , -1.1636488 , -1.5072495 ,
        1.9584545 , -1.9162863 , -2.1444912 ,  0.509831  , -0.5923175 ,
        1.1411152 ,  0.35486287, -2.6503918 , -0.46499392,  1.1868489 ,
       -0.19884808,  0.36179274, -0.12096198,  0.2726186 , -0.25528422,
        1.2648265 , -1.9108562 , -0.12047351, -1.5897545 ,  0.01586304,
        0.663553  ,  1.7322485 ,  0.22032511,  1.3360798 ,  0.33438033,
       -0.3017531 , -0.0253778 , -1.0192655 , -0.41056988,  0.13

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [40]:
wv.vectors

array([[ 2.1037366 ,  0.27816522, -0.38869938, ...,  1.1643517 ,
        -0.30108777, -0.04854615],
       [ 2.8462138 ,  0.3505207 ,  0.8030482 , ...,  0.3552366 ,
        -1.5622324 , -0.84429383],
       [ 1.3286166 ,  0.4504618 ,  2.465952  , ...,  0.5658152 ,
        -1.9536579 , -1.8359433 ],
       ...,
       [ 0.05640446, -0.05647561,  0.06413055, ...,  0.03711458,
         0.05464109, -0.01394944],
       [ 0.00851504, -0.04844569,  0.00845483, ...,  0.04753649,
        -0.00979615,  0.0896221 ],
       [-0.01342701, -0.02266734,  0.028216  , ...,  0.02154133,
        -0.01449226, -0.08784375]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [41]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.6105431318283081),
 ('princess', 0.5945087671279907),
 ('darling', 0.5573732852935791),
 ('monster', 0.5535244941711426),
 ('whir', 0.5441396236419678),
 ('theme', 0.5441322326660156),
 ('crown', 0.5419940948486328),
 ('revival', 0.5391143560409546),
 ('villain', 0.5352394580841064),
 ('reminiscent', 0.532366156578064)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!